In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
import sys, argparse, logging
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass
import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed

In [ ]:
client = MongoClient(MONGO_CONNSTR)

In [ ]:
db = client[MONGO_DB]


In [ ]:
data = db[SRC_INTRADAY].find_one({'tckr': 'US.F.CL.Q12.120720', 'dt': pd.Timestamp('2012-07-16 00:00:00.000Z')})
df = pickle.loads(data['ohlc'])

In [ ]:
tz = pytz.timezone("US/Pacific")
df = df.tz_convert(tz)

In [ ]:
tz.zone

In [ ]:
d = tz.localize(pd.Timestamp('2012-07-16 16:59:00.000'))


In [ ]:
len(df)

In [ ]:
%load_ext line_profiler
import line_profiler
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
from Cython.Compiler import Options

directive_defaults = Options.get_directive_defaults()
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [ ]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
cimport numpy as np
import cython
import numpy as np

DTYPE = np.float
ctypedef np.float64_t DTYPE_t
import pandas as pd


@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def get_quote_fast(dfg, timestamps_list):

    cdef DTYPE_t _last_px
    cdef np.uint64_t[:] npdatetime = dfg.index.values.astype('datetime64[s]').view(np.uint64)

    cdef np.ndarray[DTYPE_t, ndim=2] data = dfg.data

    cdef int ic = dfg.cols['c']
    cdef int count = data.shape[1]
    cdef int i = 0
    cdef int ts_idx = 0
    values = []
    
    if len(timestamps_list) == 0:
        return []
    
    dates = []
    for t in timestamps_list:
        # Make sure that timezones are equal
        if t.tzinfo.zone != dfg.tz.zone:
            # TODO: change to argument error
            raise ValueError("timestamps_list's and quotes dataframe timezones are inequal")
        
        dates.append(np.datetime64(t.replace(tzinfo=None)))
    
    # Create Numpy sorted dates array
    cdef np.uint64_t[:] target_timestamps = np.array(dates, dtype='datetime64[s]').view(np.uint64)
        

    cdef np.ndarray[np.int64_t, ndim=1] ts_indexes = np.searchsorted(npdatetime, target_timestamps)
    
    return ts_indexes       



In [ ]:
%lprun -f get_quote_fast get_quote_fast(DataFrameGetter(df), [tz.localize(pd.Timestamp('2012-07-15 16:59:00.000'))])

In [ ]:
%%timeit
data = db[SRC_INTRADAY].find_one({'tckr': 'US.F.CL.Q12.120720', 'dt': pd.Timestamp('2012-07-16 00:00:00.000Z')})
df = pickle.loads(data['ohlc'])
df = df.tz_convert(tz)
get_quote_fast(DataFrameGetter(df), [tz.localize(pd.Timestamp('2012-07-16 16:59:00.000')),
                                     tz.localize(pd.Timestamp('2012-07-17 16:59:00.000'))])

In [ ]:
%timeit df.index.values.astype('datetime64[s]')

In [ ]:
dfg  = DataFrameGetter(df)

In [ ]:
df.index

In [ ]:
v = []
v.append(np.datetime64(tz.localize(pd.Timestamp('2012-07-15 16:59:00.000')).replace(tzinfo=None)))
v.append(np.datetime64(tz.localize(pd.Timestamp('2012-07-15 16:39:00.000')).replace(tzinfo=None)))

In [ ]:
np.sort(np.array(v, dtype='datetime64[s]')).view(np.uint64)

In [ ]:
np.searchsorted(df.index.values, tz.localize(pd.Timestamp('2012-07-15 16:59:00.000')).value)

In [ ]:
t.date()